In [1]:
import requests
import m3u8
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get URL and output file name from environment variables
url = os.getenv("DOWNLOAD_URL")
output_file = os.getenv("OUTPUT_FILE")

# Fetch the M3U8 playlist
response = requests.get(url)
m3u8_master = m3u8.loads(response.text)
segments = m3u8_master.data["segments"]


# Function to get the size of the already downloaded file
def get_downloaded_size(filename):
    if os.path.exists(filename):
        return os.path.getsize(filename)
    return 0


# Get the size of the already downloaded file
downloaded_size = get_downloaded_size(output_file)


# Function to estimate the number of segments already downloaded
def estimate_last_segment_index(filename, segments):
    if not os.path.exists(filename) or len(segments) == 0:
        return -1

    with open(filename, "rb") as f:
        downloaded_data = f.read()

    estimated_index = -1
    for i, segment in enumerate(segments):
        segment_url = segment["uri"].replace("//", "https://")
        segment_data = requests.get(segment_url).content

        if segment_data in downloaded_data:
            estimated_index = i
        else:
            break

    return estimated_index


# Calculate the last downloaded segment index based on the file content
last_downloaded_index = estimate_last_segment_index(output_file, segments)

# Open the output file in append mode
with open(output_file, "ab") as file:
    for i, segment in enumerate(segments):
        if i <= last_downloaded_index:
            continue  # Skip already downloaded segments

        segment_url = segment["uri"]
        segment_url = segment_url.replace("//", "https://")

        try:
            r = requests.get(segment_url)
            r.raise_for_status()  # Raise an error for bad status codes
            file.write(r.content)

            # Update in-memory progress
            downloaded_size += len(r.content)
            last_downloaded_index = i

            print(f"Successfully downloaded segment {i}")
        except requests.exceptions.RequestException as e:
            print(f"Failed to download segment {i}: {e}")
            break

print("Download completed or stopped.")

Successfully downloaded segment 0
Successfully downloaded segment 1
Successfully downloaded segment 2
Successfully downloaded segment 3
Successfully downloaded segment 4
Successfully downloaded segment 5
Successfully downloaded segment 6
Successfully downloaded segment 7
Successfully downloaded segment 8
Successfully downloaded segment 9
Successfully downloaded segment 10
Successfully downloaded segment 11
Successfully downloaded segment 12
Successfully downloaded segment 13
Successfully downloaded segment 14
Successfully downloaded segment 15


KeyboardInterrupt: 